In [1]:
import numpy as np
import matplotlib.pyplot as plt


%load_ext autoreload
%autoreload 2
%matplotlib inline

#this file has all my utility functions -- although I'm reproducing the important ones below
from glm_utils import *

/home/amy/.local/lib/python2.7/site-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
/home/amy/.local/lib/python2.7/site-packages/sklearn/qda.py:4: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)


In [11]:
#magic numbers 6 and 9 come from cross validation to find best possible lag and offet. Note that was done for the natural 
#scenes stimulus so might not be the best here.

def arrange_data_glm(dff_traces, images, stim_table):
	#declare a dictionary of empty lists for each cell trace, 
	#and a list for the stimulus
	data = []
	stim_array = []
	im_array = []

	#average each trace over the presentation of each stimulus
	for index, row in stim_table.iterrows():
	    stim_array.append(images[row['frame']])
	    im_array.append(row['frame'])

	    try:
	    	data.append(np.mean(dff_traces[:, row['start'] + 6 :row['start'] + 9], axis = 1)) #/ np.mean(dff_traces[:, row['start'] : row['start'] - 30], axis = 1) )
	    except:
	    	data.append(np.mean(dff_traces[:, row['start']+ 6:row['end']], axis = 1))
	stim_array = np.array(stim_array)
	#stim_array = stim_array[:, 0:10]

	data = np.array(data)

	return data, stim_array, im_array

def download_data(region, cre_line, stimulus = None):
	'''
	region = [reg1, reg2, ...]
	cre_line = [line1, line2, ...]
	'''
	boc = BrainObservatoryCache(manifest_file='boc/manifest.json')
	ecs = boc.get_experiment_containers(targeted_structures=region, cre_lines=cre_line)

	ec_ids = [ ec['id'] for ec in ecs ]

	exp = boc.get_ophys_experiments(experiment_container_ids=ec_ids)

	if stimulus == None:
		exp = boc.get_ophys_experiments(experiment_container_ids=ec_ids)

	else:
		exp = boc.get_ophys_experiments(experiment_container_ids=ec_ids, stimuli = stimulus)


	exp_id_list = [ec['id'] for ec in exp]

	data_set = {exp_id:boc.get_ophys_experiment_data(exp_id) for exp_id in exp_id_list}

	return data_set 

def get_data(data_set, stimulus):
	'''
	returns dff, images, stim_table
	'''

	time, dff_traces = data_set.get_dff_traces()

	try:
		images = data_set.get_stimulus_template(stimulus)
	except:
		print "No stimulus template..."
		images = None

	stim_table = data_set.get_stimulus_table(stimulus)

	return dff_traces, images, stim_table

In [12]:
boc = BrainObservatoryCache(manifest_file='boc/manifest.json')

regions = ['VISp']#, 'VISpm', 'VISl', 'VISal']
lines =  ['Cux2-CreERT2']#, 'Rbp4-Cre', 'Rorb-IRES2-Cre'] 


#this dataset just has the locally sparse noise data from layer 2/3 of primary visual cortex
data_set = download_data(regions, lines, [stim_info.LOCALLY_SPARSE_NOISE])

In [13]:
#these are the IDs of the different datasets that have locally sparse noise 
#stimulus data, in layer 2/3 of primary visual cortex

data_set.keys()

[501773889,
 504642019,
 502634578,
 501337989,
 501717543,
 510174759,
 501474098,
 501254258,
 502974807,
 505143581,
 500855614,
 509644421]

In [ ]:
from allensdk.brain_observatory.locally_sparse_noise import LocallySparseNoise


#here we get the RF's that allen computed, the stimulus images, and the dff data. 
#This takes a while to run, which is why I'm only doing it for one of the datasets 
#(which you can roughly think of as being a single mouse)

key = data_set.keys()[0]

#the line below literally takes forever, only uncomment this if you really care about the lsn stimulus
lsn  = LocallySparseNoise(data_set[key])

dff, images, stim_table = get_data(data_set[key], stim_info.LOCALLY_SPARSE_NOISE)
   

In [ ]:
#here we plot the receptive fields which AIBS computed

n_cells = lsn.receptive_field.shape[2]

fig = plt.figure(figsize = [10, 100] )
i = 1
for n in range(n_cells):
    ax = plt.subplot(np.ceil(n_cells/ 2.) + 1, 2, i)

    rf = np.concatenate((lsn.receptive_field[:,:,i -1, 0], np.zeros([lsn.receptive_field.shape[0], 10]), lsn.receptive_field[:,:,i -1, 1]), axis = 1)

    plt.imshow(rf, 'PuRd')
    plt.axis('off')

    i+=1


In [ ]:
#here's where we get the tensors which are useful for fitting models

data, stim_array, _ = arrange_data_glm(dff, images, stim_table)

In [ ]:
#data is n_timepoints by n_neurons. stim_array is n_timepoints by im_y by im_x

print data.shape, stim_array.shape

In [ ]:
plt.imshow(stim_array[0], cmap = 'Greys')